In [1]:
from SpacetimeVar import *

In [2]:
DiffOrder = 4
nghostcells = 4
KOsigma = 0.1

In [3]:
dim = 2
path = "../Source/KleinGordon/"

In [4]:
fileRHS = open(path+"ET_Integration_Rhs_K.H", "w+")

In [5]:
RHS_Header = """#ifndef ET_INTEGRATION_RHS_K_H 
#define ET_INTEGRATION_RHS_K_H 

#include <AMReX_REAL.H> 
#include <AMReX_Array4.H> 
#include <ET_Integration_Setup.H> 

AMREX_GPU_DEVICE 
inline 
void 
state_rhs(int i, int j, int k, 
        amrex::Array4<amrex::Real> const& rhs_fab, 
        amrex::Array4<amrex::Real const> const& state_fab, 
        amrex::GpuArray<amrex::Real,AMREX_SPACEDIM> const& dx, 
        const amrex::GeometryData& geom) noexcept 
{
        const auto domain_xlo = geom.ProbLo(); 

        amrex::Real x0 = (i + 0.5)*geom.CellSize(0) + domain_xlo[0]; 
        amrex::Real x1 = (j + 0.5)*geom.CellSize(1) + domain_xlo[1];
        
"""

fileRHS.write(RHS_Header)

643

In [6]:
stVar.decl = []

Psi = stVar('Psi', declare = True)
Pi = stVar('Pi', declare = True)

fileRHS.write(Psi.AMReXDeclare())
fileRHS.write(Pi.AMReXDeclare())

54

In [7]:
fileVARS = open(path+"ET_Integration_Variables.H","w+")
fileVARS.write("names = {")
for itr in range(len(stVar.decl)-1):
    fileVARS.write("\""+stVar.decl[itr]+"\", ")
fileVARS.write("\""+stVar.decl[len(stVar.decl)-1]+"\"")
fileVARS.write("};")
fileVARS.close()

In [8]:
fileRHS.write(DstVar(Psi, 1, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(Psi, 2, orderD = DiffOrder).AMReXDeclare())
fileRHS.write(DstVar(Psi, 'KO', orderD = DiffOrder, sigma = KOsigma).AMReXDeclare())

879

In [9]:
x = stVarRank1('x', dim = 2)

In [10]:
m = stVar('m')
m.var = 1
fileRHS.write(m.AMReXReal())

27

In [11]:
RHS_Psi = stVar('Psi')

RHS_Psi.var = Pi.symb

In [12]:
RHS_Pi = stVar('Pi')

for i in range(dim):
    RHS_Pi.var += Dsymb(Psi.symb, str(i)+str(i))

RHS_Pi.var += -m.symb**2*Psi.symb

In [13]:
fileRHS.write(RHS_Psi.AMReXRHS())
fileRHS.write(RHS_Pi.AMReXRHS())

81

In [14]:
fileRHS.write("}\n")
fileRHS.write("#endif")
fileRHS.close()

In [15]:
fileSETUP = open(path+"ET_Integration_Setup.H", "w+")

fileSETUP.write("#ifndef ET_INTEGRATION_SETUP_K_H \n")
fileSETUP.write("#define ET_INTEGRATION_SETUP_K_H \n\n")

fileSETUP.write("#include <AMReX_REAL.H> \n")
fileSETUP.write("#include <AMReX_Array4.H> \n\n")
    
fileSETUP.write("namespace Idx { \n")
fileSETUP.write("         enum ETIndexes {")
    
Idx_string = ""
for itr in stVar.decl:
    Idx_string += itr+", "
Idx_string += "NumScalars"
    
fileSETUP.write(Idx_string)
fileSETUP.write("}; \n};\n\n")
    
fileSETUP.write("namespace Diag { \n")
fileSETUP.write("         enum DiagnosticIndexes {")
    
Idx_string = ""
for itr in stVar.declDiag:
    Idx_string += itr+", "
Idx_string += "NumScalars"
    
fileSETUP.write(Idx_string)
fileSETUP.write("}; \n};\n\n")
    
fileSETUP.write("#define NUM_GHOST_CELLS "+str(nghostcells)+"\n\n")
fileSETUP.write("#endif")

fileSETUP.close()

In [16]:
filePOST = open(path+"ET_Integration_Post_Update_K.H", "w+")

PostUpdate_Header = """#ifndef ET_INTEGRATION_POST_UPDATE_K_H 
#define ET_INTEGRATION_POST_UPDATE_K_H 

#include <AMReX_REAL.H> 
#include <AMReX_Array4.H> 
#include <ET_Integration_Setup.H> 

AMREX_GPU_DEVICE 
inline 
void 
state_post_update(int i, int j, int k, 
        amrex::Array4<amrex::Real> const& state_fab, 
        const amrex::GeometryData& geom) noexcept 
{
"""

filePOST.write(PostUpdate_Header)

filePOST.write("}\n")
filePOST.write("#endif")
filePOST.close()

In [24]:
fileINIT = open(path+"ET_Integration_Init_K.H", "w+")

Init_Header = """#ifndef ET_INTEGRATION_INIT_K_H 
#define ET_INTEGRATION_INIT_K_H 

#include <AMReX_REAL.H> 
#include <AMReX_Array4.H> 
#include <ET_Integration_Setup.H> 

AMREX_GPU_DEVICE 
inline 
void 
state_init(int i, int j, int k, 
        amrex::Array4<amrex::Real> const& state_fab, 
        amrex::Real time, const amrex::GeometryData& geom) noexcept 
{

        const auto domain_xlo = geom.ProbLo(); 

        amrex::Real x0 = (i + 0.5)*geom.CellSize(0) + domain_xlo[0]; 
        amrex::Real x1 = (j + 0.5)*geom.CellSize(1) + domain_xlo[1]; 

"""

fileINIT.write(Init_Header)

536

In [25]:
PsiInit = stVar('Psi')
PiInit = stVar('Pi')

In [26]:
PsiInit.var = sp.exp(-x.symb[0]**2 - x.symb[1]**2)
PiInit.var = 0

In [27]:
fileINIT.write(PsiInit.AMReXInit())
fileINIT.write(PiInit.AMReXInit())

42

In [28]:
fileINIT.write("}\n")
fileINIT.write("#endif")
fileINIT.close()

In [31]:
stVar.declDiag = []

Psi = stVar('Psi',declareDiag = True)

fileDIAGVARS = open(path+"ET_Integration_Diagnostic_Variables.H","w+")
fileDIAGVARS.write("names = {")
for itr in range(len(stVar.declDiag)-1):
    fileDIAGVARS.write("\""+stVar.declDiag[itr]+"\", ")
fileDIAGVARS.write("\""+stVar.declDiag[len(stVar.declDiag)-1]+"\"")
fileDIAGVARS.write("};")
fileDIAGVARS.close()

In [33]:
fileDIAG = open(path+"ET_Integration_Diag_K.H", "w+")

DIAG_Header = """#ifndef ET_INTEGRATION_DIAG_K_H 
#define ET_INTEGRATION_DIAG_K_H 

#include <AMReX_REAL.H> 
#include <AMReX_Array4.H> 
#include <ET_Integration_Setup.H> 

AMREX_GPU_DEVICE 
inline 
void 
state_diagnostics(int i, int j, int k, 
        amrex::Array4<amrex::Real> const& diag, 
        amrex::Array4<amrex::Real const> const& state_fab, 
        amrex::GpuArray<amrex::Real,AMREX_SPACEDIM> const& dx,
        const amrex::GeometryData& geom) noexcept 
{

        const auto domain_xlo = geom.ProbLo(); 

        amrex::Real x0 = (i + 0.5)*geom.CellSize(0) + domain_xlo[0]; 
        amrex::Real x1 = (j + 0.5)*geom.CellSize(0) + domain_xlo[1]; 
        amrex::Real x2 = (k + 0.5)*geom.CellSize(0) + domain_xlo[2];
"""

fileDIAG.write(DIAG_Header)

711

In [34]:
fileDIAG.write("}\n")
fileDIAG.write("#endif")
fileDIAG.close()